A player's season momentum rating is defined as:


      M = exceeded_weekly_projection/total_games
      
Limitations:
 - if a player got injured during the game
 - if a player was dealing with an injury that week, but still played
 - bad weather (**)

# Preprocess 2019, 2020, & 2021 data

In [1]:
import pandas as pd
import numpy as np

In [2]:
actual_2019 = pd.read_csv('../data/fntsy-data_2019_ffb-weekly-stats_reg-season_dkng.csv')
projected_2019 = pd.read_csv('../data/fntsy-data_2019_ffb-weekly-projections_reg-season_dkng.csv')
actual_2020 = pd.read_csv('../data/fntsy-data_2020_ffb-weekly-stats_reg-season_dkng.csv')
projected_2020 = pd.read_csv('../data/fntsy-data_2020_ffb-weekly-projections_reg-season_dkng.csv')
actual_2021 = pd.read_csv('../data/fntsy-data_2021_ffb-week1-to-week5-stats_reg-season_dkng.csv')
projected_2021 = pd.read_csv('../data/fntsy-data_2021_ffb-week1-to-week5-projections_reg-season-dkng.csv')

In [3]:
# manually adjust 2020 columns from 'FantasyPointsPerGame' to 
# FantastyPointsPerGameDraftKings for consistency with other csv files
projected_2020 = projected_2020.rename(columns={'FantasyPointsPerGame': 'FantasyPointsPerGameDraftKings'})
actual_2020 = actual_2020.rename(columns={'FantasyPointsPerGame': 'FantasyPointsPerGameDraftKings'})

In [4]:
def df_filter(df):
    '''Select only columns of interest, sort data, rename columns.'''
    
    df = df[['Week','Name', 'Team', 'Position', 'Opponent', 'FantasyPointsPerGameDraftKings']]
    df = df.sort_values(by=['Week', 'FantasyPointsPerGameDraftKings', 'Name', 'Position'],
                          ascending = [True, False, True, True])
    df = df.rename(columns={'Position': 'POS', 'Opponent': 'OPP', 'FantasyPointsPerGameDraftKings': 'FPPG'})
    return df

In [5]:
actual_2019 = df_filter(actual_2019)
projected_2019 = df_filter(projected_2019)
actual_2020 = df_filter(actual_2020)
projected_2020 = df_filter(projected_2020)
actual_2021 = df_filter(actual_2021)
projected_2021 = df_filter(projected_2021)

In [6]:
actual_2019 = actual_2019.add_prefix('a2019_')
projected_2019 = projected_2019.add_prefix('p2019_')
actual_2020 = actual_2020.add_prefix('a2020_')
projected_2020 = projected_2020.add_prefix('p2020_')
actual_2021 = actual_2021.add_prefix('a2021_')
projected_2021 = projected_2021.add_prefix('p2021_')

In [7]:
merged_2019 = projected_2019.merge(actual_2019, how='outer', left_on=['p2019_Name', 'p2019_Week'],
                                   right_on=['a2019_Name', 'a2019_Week'], indicator=True)
merged_2020 = projected_2020.merge(actual_2020, how='outer', left_on=['p2020_Name', 'p2020_Week'],
                                   right_on=['a2020_Name', 'a2020_Week'], indicator=True)
merged_2021 = projected_2021.merge(actual_2021, how='outer', left_on=['p2021_Name', 'p2021_Week'],
                                   right_on=['a2021_Name', 'a2021_Week'], indicator=True)

In [8]:
merged_2019 = merged_2019[['p2019_Week', 'p2019_Name', 'p2019_Team', 'p2019_POS', 'p2019_OPP',
                      'p2019_FPPG', 'a2019_FPPG', '_merge']]
merged_2019 = merged_2019.rename(columns={'p2019_Week': '2019_Week', 'p2019_Name': '2019_Name',
                                          'p2019_POS': '2019_POS', 'p2019_OPP': 'OPP'})
merged_2020 = merged_2020[['p2020_Week', 'p2020_Name', 'p2020_Team', 'p2020_POS', 'p2020_OPP',
                      'p2020_FPPG', 'a2020_FPPG', '_merge']]
merged_2020 = merged_2020.rename(columns={'p2020_Week': '2020_Week', 'p2020_Name': '2020_Name',
                                          'p2020_POS': '2020_POS', 'p2020_OPP': 'OPP'})
merged_2021 = merged_2021[['p2021_Week', 'p2021_Name', 'p2021_Team', 'p2021_POS', 'p2021_OPP',
                      'p2021_FPPG', 'a2021_FPPG', '_merge']]
merged_2021 = merged_2021.rename(columns={'p2021_Week': '2021_Week', 'p2021_Name': '2021_Name',
                                          'p2021_POS': '2021_POS', 'p2021_OPP': 'OPP'})

In [9]:
merged_2019 = merged_2019[merged_2019['2019_POS'].isin(['RB', 'WR', 'QB', 'TE', 'DST'])]

In [10]:
merged_2020 = merged_2020[merged_2020['2020_POS'].isin(['RB', 'WR', 'QB', 'TE', 'DST'])]

In [11]:
merged_2021 = merged_2021[merged_2021['2021_POS'].isin(['RB', 'WR', 'QB', 'TE', 'DST'])]

In [12]:
merged_2019 = merged_2019.dropna()

In [13]:
merged_2020 = merged_2020.dropna()

In [14]:
merged_2021 = merged_2021.dropna()

# Momentum

In [15]:
def momentum(df, season):
    df['delta'] = df[f'a{season}_FPPG'] - df[f'p{season}_FPPG']
    df['pos_delta'] = df.apply(lambda x: 1 if x.delta > 0 else 0, axis=1)
    avg_delta = df.groupby(f'{season}_Name').apply(lambda x: x.delta.mean()).reset_index()
    momentum_scores = df.groupby(f'{season}_Name').apply(lambda x: x.pos_delta.sum()/len(x)).reset_index()
    games_played = df.groupby(f'{season}_Name').apply(lambda x: len(x)).reset_index()
    season_pos_delta = df.groupby(f'{season}_Name').apply(lambda x: x.pos_delta.sum()).reset_index()
    momentum = season_pos_delta.merge(games_played, on=f'{season}_Name')
    momentum = momentum.rename(columns={'0_x': f'games_exceeded_{season}', 
                              '0_y': f'games_played_{season}'})

    momentum = momentum.merge(momentum_scores, on=f'{season}_Name')
    momentum = momentum.rename(columns={0: 'momentum'})
    momentum = momentum.merge(avg_delta, on=f'{season}_Name')
    momentum = momentum.rename(columns={f'{season}_Name': f'Name_{season}', 0: f'avg_delta_{season}'})
    
    return momentum

# 2019 Calculation

In [16]:
momentum_2019 = momentum(merged_2019, 2019)

# 2020 Calculation

In [17]:
momentum_2020 = momentum(merged_2020, 2020)

# 2021 Week 1 to Week 5 Calculation

In [18]:
momentum_2021 = momentum(merged_2021, 2021)

# Momentum Merge and Multiplier Calculation

In [20]:
momentum_merged = momentum_2021.merge(momentum_2020, how='left', left_on='Name_2021', right_on='Name_2020', indicator='21_20_merge')

In [21]:
momentum_merged = momentum_merged.merge(momentum_2019, how='left', left_on='Name_2021', right_on='Name_2019', indicator='21_19_merge')

In [22]:
momentum_merged.columns

Index(['Name_2021', 'games_exceeded_2021', 'games_played_2021', 'momentum_x',
       'avg_delta_2021', 'Name_2020', 'games_exceeded_2020',
       'games_played_2020', 'momentum_y', 'avg_delta_2020', '21_20_merge',
       'Name_2019', 'games_exceeded_2019', 'games_played_2019', 'momentum',
       'avg_delta_2019', '21_19_merge'],
      dtype='object')

In [23]:
# fill numerical values with zero if NaN
momentum_merged[['games_exceeded_2021', 'games_played_2021', 'avg_delta_2021', 
                 'games_exceeded_2020', 'games_played_2020', 'avg_delta_2020',
                 'games_exceeded_2019', 'games_played_2019', 'avg_delta_2019'
                ]] = momentum_merged[['games_exceeded_2021', 'games_played_2021', 'avg_delta_2021', 
                 'games_exceeded_2020', 'games_played_2020', 'avg_delta_2020',
                 'games_exceeded_2019', 'games_played_2019', 'avg_delta_2019'
                ]].fillna(0)
momentum_merged

,Name_2021,games_exceeded_2021,games_played_2021,momentum_x,avg_delta_2021,Name_2020,games_exceeded_2020,games_played_2020,momentum_y,avg_delta_2020,21_20_merge,Name_2019,games_exceeded_2019,games_played_2019,momentum,avg_delta_2019,21_19_merge
0,A.J. Brown,0,4,0.0,-9.725000,A.J. Brown,10.0,11.0,0.909091,10.836364,both,A.J. Brown,6.0,7.0,0.857143,10.157143,both
1,A.J. Green,3,6,0.5,-0.516667,NaN,0.0,0.0,NaN,0.000000,left_only,NaN,0.0,0.0,NaN,0.000000,left_only
2,AJ Dillon,2,5,0.4,1.840000,NaN,0.0,0.0,NaN,0.000000,left_only,NaN,0.0,0.0,NaN,0.000000,left_only
3,Aaron Jones,1,5,0.2,0.520000,Aaron Jones,8.0,14.0,0.571429,4.800000,both,Aaron Jones,8.0,12.0,0.666667,9.983333,both
4,Aaron Rodgers,3,5,0.6,-1.540000,Aaron Rodgers,13.0,15.0,0.866667,6.206667,both,Aaron Rodgers,6.0,16.0,0.375000,-0.200000,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,Zach Ertz,2,5,0.4,-0.980000,NaN,0.0,0.0,NaN,0.000000,left_only,Zach Ertz,7.0,10.0,0.700000,4.540000,both
405,Zach Pascal,2,5,0.4,1.660000,NaN,0.0,0.0,NaN,0.000000,left_only,Zach Pascal,3.0,3.0,1.000000,9.400000,both
406,Zach Wilson,2,5,0.4,-4.100000,NaN,0.0,0.0,NaN,0.000000,left_only,NaN,0.0,0.0,NaN,0.000000,left_only
407,Zack Moss,4,4,1.0,6.550000,NaN,0.0,0.0,NaN,0.000000,left_only,NaN,0.0,0.0,NaN,0.000000,left_only


In [24]:
# cumulative games played
momentum_merged['cum_games'] = momentum_merged.games_played_2021 + momentum_merged.games_played_2020 + \
                               momentum_merged.games_played_2019

In [25]:
# cumulative exceeded
momentum_merged['cum_exceeded'] = momentum_merged.games_exceeded_2021 + momentum_merged.games_exceeded_2020 + \
                               momentum_merged.games_exceeded_2019

In [26]:
# cumulative momentum
momentum_merged['cum_momentum'] = momentum_merged.cum_exceeded / momentum_merged.cum_games

In [27]:
# cumulative average delta
momentum_merged['cum_avg_delta'] = (momentum_merged.avg_delta_2021 * momentum_merged.games_played_2021 + \
                                momentum_merged.avg_delta_2020 * momentum_merged.games_played_2020 + \
                                momentum_merged.avg_delta_2021 * momentum_merged.games_played_2021) / \
                                momentum_merged.cum_games

In [28]:
# multiplier calculation
# [0, 0.25) --> 0.9
# [0.25, 0.5) --> 0.95
# 0.5 --> 1.0
# (0.5, 0.75) --> 1.05
# [0.75, 1.0] --> 1.10

def multiplier(momentum):
    if momentum < 0.25:
        m = 0.9
    elif momentum < 0.5:
        m = 0.95
    elif momentum == 0.5:
        m = 1.0
    elif momentum < 0.75:
        m = 1.05
    else:
        m = 1.1
    return m
        
momentum_merged['multiplier'] = momentum_merged.cum_momentum.apply(lambda x: multiplier(x))

In [29]:
momentum_merged

,Name_2021,games_exceeded_2021,games_played_2021,momentum_x,avg_delta_2021,Name_2020,games_exceeded_2020,games_played_2020,momentum_y,avg_delta_2020,...,games_exceeded_2019,games_played_2019,momentum,avg_delta_2019,21_19_merge,cum_games,cum_exceeded,cum_momentum,cum_avg_delta,multiplier
0,A.J. Brown,0,4,0.0,-9.725000,A.J. Brown,10.0,11.0,0.909091,10.836364,...,6.0,7.0,0.857143,10.157143,both,22.0,16.0,0.727273,1.881818,1.05
1,A.J. Green,3,6,0.5,-0.516667,NaN,0.0,0.0,NaN,0.000000,...,0.0,0.0,NaN,0.000000,left_only,6.0,3.0,0.500000,-1.033333,1.00
2,AJ Dillon,2,5,0.4,1.840000,NaN,0.0,0.0,NaN,0.000000,...,0.0,0.0,NaN,0.000000,left_only,5.0,2.0,0.400000,3.680000,0.95
3,Aaron Jones,1,5,0.2,0.520000,Aaron Jones,8.0,14.0,0.571429,4.800000,...,8.0,12.0,0.666667,9.983333,both,31.0,17.0,0.548387,2.335484,1.05
4,Aaron Rodgers,3,5,0.6,-1.540000,Aaron Rodgers,13.0,15.0,0.866667,6.206667,...,6.0,16.0,0.375000,-0.200000,both,36.0,22.0,0.611111,2.158333,1.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,Zach Ertz,2,5,0.4,-0.980000,NaN,0.0,0.0,NaN,0.000000,...,7.0,10.0,0.700000,4.540000,both,15.0,9.0,0.600000,-0.653333,1.05
405,Zach Pascal,2,5,0.4,1.660000,NaN,0.0,0.0,NaN,0.000000,...,3.0,3.0,1.000000,9.400000,both,8.0,5.0,0.625000,2.075000,1.05
406,Zach Wilson,2,5,0.4,-4.100000,NaN,0.0,0.0,NaN,0.000000,...,0.0,0.0,NaN,0.000000,left_only,5.0,2.0,0.400000,-8.200000,0.95
407,Zack Moss,4,4,1.0,6.550000,NaN,0.0,0.0,NaN,0.000000,...,0.0,0.0,NaN,0.000000,left_only,4.0,4.0,1.000000,13.100000,1.10


In [30]:
# filter out multiplier for players less than 5 total games played
momentum_merged = momentum_merged[momentum_merged.cum_games > 5]

In [31]:
momentum_merged

,Name_2021,games_exceeded_2021,games_played_2021,momentum_x,avg_delta_2021,Name_2020,games_exceeded_2020,games_played_2020,momentum_y,avg_delta_2020,...,games_exceeded_2019,games_played_2019,momentum,avg_delta_2019,21_19_merge,cum_games,cum_exceeded,cum_momentum,cum_avg_delta,multiplier
0,A.J. Brown,0,4,0.0,-9.725000,A.J. Brown,10.0,11.0,0.909091,10.836364,...,6.0,7.0,0.857143,10.157143,both,22.0,16.0,0.727273,1.881818,1.05
1,A.J. Green,3,6,0.5,-0.516667,NaN,0.0,0.0,NaN,0.000000,...,0.0,0.0,NaN,0.000000,left_only,6.0,3.0,0.500000,-1.033333,1.00
3,Aaron Jones,1,5,0.2,0.520000,Aaron Jones,8.0,14.0,0.571429,4.800000,...,8.0,12.0,0.666667,9.983333,both,31.0,17.0,0.548387,2.335484,1.05
4,Aaron Rodgers,3,5,0.6,-1.540000,Aaron Rodgers,13.0,15.0,0.866667,6.206667,...,6.0,16.0,0.375000,-0.200000,both,36.0,22.0,0.611111,2.158333,1.05
5,Adam Humphries,2,5,0.4,-1.340000,NaN,0.0,0.0,NaN,0.000000,...,0.0,1.0,0.000000,-0.300000,both,6.0,2.0,0.333333,-2.233333,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,Wayne Gallman,1,2,0.5,-0.650000,Wayne Gallman,2.0,3.0,0.666667,4.300000,...,1.0,1.0,1.000000,16.300000,both,6.0,4.0,0.666667,1.716667,1.05
401,Will Dissly,2,4,0.5,-0.575000,NaN,0.0,0.0,NaN,0.000000,...,2.0,2.0,1.000000,4.900000,both,6.0,4.0,0.666667,-0.766667,1.05
402,William Fuller V,0,2,0.0,-5.800000,William Fuller V,7.0,7.0,1.000000,12.257143,...,2.0,6.0,0.333333,7.683333,both,15.0,9.0,0.600000,4.173333,1.05
404,Zach Ertz,2,5,0.4,-0.980000,NaN,0.0,0.0,NaN,0.000000,...,7.0,10.0,0.700000,4.540000,both,15.0,9.0,0.600000,-0.653333,1.05


In [32]:
momentum_merged[momentum_merged.Name_2021 == 'Leonard Fournette']

,Name_2021,games_exceeded_2021,games_played_2021,momentum_x,avg_delta_2021,Name_2020,games_exceeded_2020,games_played_2020,momentum_y,avg_delta_2020,...,games_exceeded_2019,games_played_2019,momentum,avg_delta_2019,21_19_merge,cum_games,cum_exceeded,cum_momentum,cum_avg_delta,multiplier
254,Leonard Fournette,4,5,0.8,0.94,Leonard Fournette,3.0,3.0,1.0,4.5,...,4.0,15.0,0.266667,-0.22,both,23.0,11.0,0.478261,0.995652,0.95


In [33]:
momentum_merged.to_csv('momentum.csv')

0.5 and above is good -> m > 1.0
consider only players who have a sample size > 5

somehow factor in by HOW MUCH a player outperforms on average (see above code snippet for CMC)